ETAPA DE PRÉ-PROCESSAMENTO DOS DADOS

Os dados utilizados foram retirados da base de dados abertos da Polícia Federal.

Os dados podem ser obtidos através do link: https://servicos.dpf.gov.br/dadosabertos/SINARM_CSV/OCORRENCIAS/

Importação de bibliotecas

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import chardet

Carregando os dados

In [6]:
#Encontrar o tipo de formatação dos dados

with open("OCORRENCIAS_2024.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # Lê os primeiros 100.000 bytes
    print(result["encoding"])

ISO-8859-1


In [15]:
#Transformar o padrão de codificação dos caracteres

with open("OCORRENCIAS_2024.csv", "r", encoding="ISO-8859-1") as f:
    conteudo = f.read()

with open("OCORRENCIAS_2024_utf8.csv", "w", encoding="utf-8") as f:
    f.write(conteudo)


In [21]:
#Carregamento dos dados
dados = pd.read_csv("OCORRENCIAS_2024_utf8.csv", encoding="utf-8", sep=";") #Separador dos dados ";"